In [1]:
"""
SHERPA is a Python library for hyperparameter tuning of machine learning models.
Copyright (C) 2018  Lars Hertel, Peter Sadowski, and Julian Collado.

This file is part of SHERPA.

SHERPA is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

SHERPA is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with SHERPA.  If not, see <http://www.gnu.org/licenses/>.
"""
from __future__ import print_function
import sherpa
import time
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.datasets import mnist

/Users/larshertel/python3env/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
batch_size = 32
num_classes = 10
epochs = 15

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [4]:
parameters = [sherpa.Discrete('num_units', [32, 128]),
              sherpa.Choice('activation', ['relu', 'tanh', 'sigmoid'])]

In [5]:
algorithm = sherpa.algorithms.BayesianOptimization(max_num_trials=50)

In [6]:
study = sherpa.Study(parameters=parameters,
                     algorithm=algorithm,
                     lower_is_better=True)

INFO:sherpa.core:
-------------------------------------------------------
SHERPA Dashboard running. Access via
http://192.168.2.3:8880 if on a cluster or
http://localhost:8880 if running locally.
-------------------------------------------------------


In [7]:
num_iterations = 10

for trial in study:
    print("Trial {}:\t{}".format(trial.id, trial.parameters))

    model = Sequential()
    model.add(Dense(units=trial.parameters['num_units'], activation=trial.parameters['activation'], input_dim=784))
    model.add(Dense(units=10, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              callbacks=[study.keras_callback(trial, objective_name='val_loss')],
              validation_data=(x_test, y_test))

    study.finalize(trial=trial)

Trial 1:	{'activation': 'relu', 'num_units': 64}
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 3s 51us/step - loss: 0.3142 - acc: 0.9121 - val_loss: 0.1821 - val_acc: 0.9473
Epoch 2/15
60000/60000 [==============================] - 3s 52us/step - loss: 0.1534 - acc: 0.9547 - val_loss: 0.1312 - val_acc: 0.9604
Epoch 3/15
60000/60000 [==============================] - 3s 53us/step - loss: 0.1113 - acc: 0.9673 - val_loss: 0.1086 - val_acc: 0.9675
Epoch 4/15
60000/60000 [==============================] - 3s 51us/step - loss: 0.0872 - acc: 0.9734 - val_loss: 0.1055 - val_acc: 0.9687
Epoch 5/15
60000/60000 [==============================] - 3s 51us/step - loss: 0.0706 - acc: 0.9789 - val_loss: 0.0893 - val_acc: 0.9715
Epoch 6/15
60000/60000 [==============================] - 3s 49us/step - loss: 0.0593 - acc: 0.9822 - val_loss: 0.0912 - val_acc: 0.9702
Epoch 7/15
60000/60000 [==============================] - 3s 51us/step - loss: 

60000/60000 [==============================] - 3s 52us/step - loss: 0.0132 - acc: 0.9972 - val_loss: 0.0823 - val_acc: 0.9763
Epoch 13/15
60000/60000 [==============================] - 3s 49us/step - loss: 0.0114 - acc: 0.9976 - val_loss: 0.0773 - val_acc: 0.9797
Epoch 14/15
60000/60000 [==============================] - 3s 48us/step - loss: 0.0089 - acc: 0.9982 - val_loss: 0.0849 - val_acc: 0.9771
Epoch 15/15
60000/60000 [==============================] - 3s 46us/step - loss: 0.0083 - acc: 0.9982 - val_loss: 0.0901 - val_acc: 0.9768
Trial 5:	{'activation': 'sigmoid', 'num_units': 64}
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 3s 46us/step - loss: 0.4642 - acc: 0.8862 - val_loss: 0.2509 - val_acc: 0.9287
Epoch 2/15
60000/60000 [==============================] - 2s 41us/step - loss: 0.2210 - acc: 0.9371 - val_loss: 0.1913 - val_acc: 0.9443
Epoch 3/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.171

/Users/larshertel/python3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Trial 7:	{'num_units': 128, 'activation': 'sigmoid'}
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 4s 62us/step - loss: 0.3965 - acc: 0.8965 - val_loss: 0.2219 - val_acc: 0.9351
Epoch 2/15
60000/60000 [==============================] - 3s 55us/step - loss: 0.1945 - acc: 0.9443 - val_loss: 0.1579 - val_acc: 0.9554
Epoch 3/15
60000/60000 [==============================] - 3s 54us/step - loss: 0.1427 - acc: 0.9589 - val_loss: 0.1314 - val_acc: 0.9616
Epoch 4/15
60000/60000 [==============================] - 3s 55us/step - loss: 0.1106 - acc: 0.9682 - val_loss: 0.1153 - val_acc: 0.9657
Epoch 5/15
60000/60000 [==============================] - 3s 54us/step - loss: 0.0887 - acc: 0.9747 - val_loss: 0.0979 - val_acc: 0.9711
Epoch 6/15
60000/60000 [==============================] - 3s 55us/step - loss: 0.0728 - acc: 0.9796 - val_loss: 0.0914 - val_acc: 0.9720
Epoch 7/15
60000/60000 [==============================] - 4s 61us/step - lo

/Users/larshertel/python3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


...

/Users/larshertel/python3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Trial 18:	{'num_units': 125, 'activation': 'tanh'}
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 5s 78us/step - loss: 0.2874 - acc: 0.9171 - val_loss: 0.1685 - val_acc: 0.9496
Epoch 2/15
60000/60000 [==============================] - 4s 64us/step - loss: 0.1415 - acc: 0.9591 - val_loss: 0.1144 - val_acc: 0.9643
Epoch 3/15
60000/60000 [==============================] - 4s 64us/step - loss: 0.0966 - acc: 0.9721 - val_loss: 0.1001 - val_acc: 0.9683
Epoch 4/15
60000/60000 [==============================] - 5s 77us/step - loss: 0.0709 - acc: 0.9800 - val_loss: 0.0814 - val_acc: 0.9753
Epoch 5/15
60000/60000 [==============================] - 4s 66us/step - loss: 0.0536 - acc: 0.9842 - val_loss: 0.0836 - val_acc: 0.9742
Epoch 6/15
60000/60000 [==============================] - 4s 65us/step - loss: 0.0420 - acc: 0.9878 - val_loss: 0.0773 - val_acc: 0.9762
Epoch 7/15
60000/60000 [==============================] - 4s 65us/step - loss

/Users/larshertel/python3env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([-0.00025845]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 52, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)
/Users/larshertel/python3env/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:457: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([3.33115559e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 58, 'nit': 6, 'warnflag': 2}
  " state: %s" % convergence_dict)
/Users/larshertel/python3env/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Trial 19:	{'num_units': 48, 'activation': 'tanh'}
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 3s 55us/step - loss: 0.3325 - acc: 0.9067 - val_loss: 0.1965 - val_acc: 0.9440
Epoch 2/15
60000/60000 [==============================] - 3s 44us/step - loss: 0.1726 - acc: 0.9503 - val_loss: 0.1504 - val_acc: 0.9547
Epoch 3/15
60000/60000 [==============================] - 3s 44us/step - loss: 0.1310 - acc: 0.9618 - val_loss: 0.1283 - val_acc: 0.9613
Epoch 4/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.1059 - acc: 0.9695 - val_loss: 0.1086 - val_acc: 0.9670
Epoch 5/15
60000/60000 [==============================] - 3s 46us/step - loss: 0.0891 - acc: 0.9742 - val_loss: 0.1040 - val_acc: 0.9678
Epoch 6/15
60000/60000 [==============================] - 3s 54us/step - loss: 0.0768 - acc: 0.9779 - val_loss: 0.1023 - val_acc: 0.9674
Epoch 7/15
60000/60000 [==============================] - 3s 45us/step - loss:

KeyboardInterrupt: 

In [ ]:
print(study.get_best_result())